<a href="https://colab.research.google.com/github/kamalkarki/home_credit/blob/master/hc_2_baseline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'home-credit-credit-risk-model-stability:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F50160%2F7921029%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T072848Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D754612b101088cf9888717a74973cded2e46bb1591812cbdee7ea2d6b9668c0e0db13070ac2b5ed8805764b2187b0746ed7a53ef70675d12a52a1f2e47574925cfb063ef629cb9e4c3b293de3f2f2a8f3d18e1edb8bd9b5deda8cb2ef1ff32f5759a3735594170c17dd6d9b600715da4fb7ce3a03f07d3578aa241bf5a551495693b15c3d0b397b5bdb97b1ade0b61904d4525f90f10036b64e04706e02b91903882a88e92b5f933fcaa0edf429657a3eb257ea9c837c57ca1c29dbfbfdb201165b8815e8966a6848a42bc86ee95122805f10d20dec94a7cdfc1ce489d6aabfc6fecc1b2600ff93cb1e547dcc19947b2f3aa903e820e01234a0edc32d170dc32'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 3375704785 bytes downloaded
Downloaded and uncompressed: home-credit-credit-risk-model-stability
Data source import complete.


In [30]:
! pip install -U "neptune[optuna]" optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.

In [31]:
# Note: I'm looking for a job in Europe, if you like my work don't hesitate to reach =)

import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin

import lightgbm as lgb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Pre-Fitted Voting Model

In [32]:
class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

### Pipeline

In [33]:
class Pipeline:
    @staticmethod # belongs to class , no object association
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.Categorical))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))

        return df

    @staticmethod
    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                df = df.with_columns(pl.col(col).cast(pl.Float32))

        df = df.drop("date_decision", "MONTH")

        return df

    @staticmethod
    def filter_cols(df):
        """
        Removes columns with more than 95% missing values.
        Removes columns with only one unique value.
        """
        df_95=[]
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()

                if isnull > 0.95:
                    # df = df.drop(col)
                    df_95.append(col)

        print("Columns with more than 95% null values",len(df_95))

        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.Categorical):
                freq = df[col].n_unique()

                # if (freq == 1) | (freq > 200): # commented on 28 April 24
                if (freq == 1):
                    df = df.drop(col)

        return df

### Automatic Aggregation

In [34]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

### File I/O

In [35]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)

    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)

        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

        chunks.append(df)

    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])

    return df

### Feature Engineering

In [36]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )

    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")

    df_base = df_base.pipe(Pipeline.handle_dates)

    return df_base

In [37]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()

    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)

    df_data[cat_cols] = df_data[cat_cols].astype("category")

    return df_data, cat_cols

### Configuration

In [38]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

### Train Files Read & Feature Engineering

In [39]:
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
    ]
}

<ipython-input-35-22f16d7b69a4>:21: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = pl.concat(chunks, how="vertical_relaxed")


In [40]:
df_train = feature_eng(**data_store)

print("train data shape:\t", df_train.shape)

train data shape:	 (1526659, 472)


### Test Files Read & Feature Engineering

In [41]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
    ]
}

<ipython-input-35-22f16d7b69a4>:21: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = pl.concat(chunks, how="vertical_relaxed")


In [42]:
df_test = feature_eng(**data_store)

print("test data shape:\t", df_test.shape)

test data shape:	 (10, 471)


### Feature Elimination

In [43]:
df_train = df_train.pipe(Pipeline.filter_cols)
df_test = df_test.select([col for col in df_train.columns if col != "target"])

print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

Columns with more than 95% null values 135
train data shape:	 (1526659, 428)
test data shape:	 (10, 427)


### Pandas Conversion

In [44]:
df_train, cat_cols = to_pandas(df_train)
df_test, cat_cols = to_pandas(df_test, cat_cols)

### My edit of feature engineering

In [45]:
def get_df_data_type(df):
    """
    input - data frame
    output- dictionary with data type as key and value as number of counts of that data type
    """
    data_type = {}
    for col in df.columns:
        col_type = df[col].dtype.name
        if col_type not in data_type.keys():
            data_type[col_type] = 1
        else:
            data_type[col_type] += 1
    return data_type


col_data_type = get_df_data_type(df_train)
print(col_data_type)
col_data_type = get_df_data_type(df_test)
print(col_data_type) # there is a miss match for cat type - need to be dealt below

{'int32': 3, 'int64': 1, 'int8': 2, 'float32': 56, 'float64': 308, 'category': 57, 'bool': 1}
{'int32': 2, 'int8': 2, 'float32': 56, 'float64': 309, 'category': 57, 'bool': 1}


In [46]:
# Now get list of feature which have got common description:
def common_feature_removal(df):
    """
    read base file of features

    Step 2
    From the description file of features:
    There are some features having same description, those are filtered and only one is kept as feature,
    rest are removed/dropped

    """

    feature_file_path = "/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv"
    df_feature = pd.read_csv(feature_file_path)
    print("Features loaded")
    print(f"Total number of features: {len(df_feature)}")

    #step 2
    des_key = {}
    feature_name, des  = df_feature["Variable"].values, df_feature["Description"]
    for index in range(len(des)):
        if des[index] in des_key.keys():
            des_key[des[index]].append(feature_name[index])
        else:
            des_key[des[index]]=[feature_name[index]]

    feature_to_remove = []
    for key, value in des_key.items():
        if len(value) >= 2:
            for f in value[1:]:
                feature_to_remove.append(f)

    # now drop these features from the data_frame

    for f in feature_to_remove:
        try:
            df = df.drop(f,axis=1)
        except:
            pass
    print("Feature after remove", len(df.columns))
    return df

X_train = common_feature_removal(df_train)

# def get_num_group_feature(df):
#     num_group = []
#     for col in df.columns:
#         if "num_group1" in col:
#             num_group.append(col)
#     return num_group

Features loaded
Total number of features: 465
Feature after remove 413


In [47]:

from sklearn.feature_selection import VarianceThreshold
def get_numeric_columns(df):
    numeric_columns = []
    for col in df.columns:
        col_type = df[col].dtype
        col_type = df[col].dtype.name # in one of code I have used it,check and confirm
        if str(col_type)[:3] in ["int", "flo"]:
            numeric_columns.append(col)
        else:
            # print(col_type)
            pass
    return numeric_columns

# Fit the VarianceThreshold transformer with the converted data frame
def check_numeric_variance(X_train, threshold=0):
    """
    based on a data set and a variance value filter columns
    """
    sel = VarianceThreshold(threshold)
    numeric_cols = get_numeric_columns(X_train)
    sel.fit(X_train[numeric_cols])
    sel_features = sel.get_feature_names_out()
    non_selected_feature = [x for x in numeric_cols if x not in sel_features]
    print("Not selected numric cols", len(non_selected_feature))
    # getting the unique count of not selected featues
    unique_count = {}
    for col in non_selected_feature:
        unique_count[col] = len(X_train[col].unique())
    # feature removal list
    feature_removal = non_selected_feature
    return feature_removal, unique_count

numeric_feature_removal, numeric_feature_removal_unqiue_count = check_numeric_variance(X_train)


Not selected numric cols 11


In [48]:
# train set
# first get training data- remove numeric_cols
X_train = X_train.drop(columns=numeric_feature_removal)
y = X_train["target"]
X = X_train.drop(columns=["target", "case_id", "WEEK_NUM"])
weeks = X_train["WEEK_NUM"]




In [49]:
# test set
# X_test = df_test.select([col for col in X.columns if col != "target"])
X_test = df_test[X.columns.difference(["target"])]
print(X.shape)
print(X_test.shape)

(1526659, 399)
(10, 399)


### Garbage Collection

In [50]:
del data_store

gc.collect()

47

### EDA

In [51]:
# print("Train is duplicated:\t", df_train["case_id"].duplicated().any())
# print("Train Week Range:\t", (df_train["WEEK_NUM"].min(), df_train["WEEK_NUM"].max()))

# print()

# print("Test is duplicated:\t", df_test["case_id"].duplicated().any())
# print("Test Week Range:\t", (df_test["WEEK_NUM"].min(), df_test["WEEK_NUM"].max()))

In [52]:
# sns.lineplot(
#     data=df_train,
#     x="WEEK_NUM",
#     y="target",
# )
# plt.show()

### Training

In [53]:
import lightgbm as lgb
import neptune
import neptune.integrations.optuna as optuna_utils
import optuna
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)



run = neptune.init_run(
                capture_hardware_metrics=True,
                name="base_line_2_lgbm",
                project="kaggle-housing/home-depot",
                api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhM2JlMDQzZS1jMzkzLTQwZDktYmY3YS1iM2Q0ODNiMDg1YzMifQ==",
)


from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


def objective(trial):

    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

    # param = {
    #     "verbose": -1,
    #     "n_iter": int(350/5),
    #     "objective": "binary",
    #     "metric": "auc",
    #     "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
    #     "num_leaves": trial.suggest_int("num_leaves", 2, 256),
    #     "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0, step=0.1),
    #     "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0, step=0.1),
    #     "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    # }
    param = {
        # "device": "gpu",
        "n_jobs": -1,
        "n_iter": 400,
        "verbosity": -1,
        "objective": "binary",
        "metric":"binary_logloss","auc"
        "boosting_type": "gbdt",
        # "feature_name": TRAIN_COLUMNS,
        # "categorical_feature": list(CATEGORY_COLUMNS.keys()),
        # "max_bin": 255,
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.6, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 50, 800),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 16, 512),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 4, 512),
    }
    gbm = lgb.train(param,
                    train_data,
                    valid_sets=[train_data,valid_data],
                    valid_names=['train','valid'])
    # print(gbm.best_iteration
    preds = gbm.predict(x_test)
    accuracy = roc_auc_score(y_test, preds)

    return accuracy

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kaggle-housing/home-depot/e/HOM-19
[neptune] [warning] Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/help/nvml_error/


In [54]:


neptune_callback = optuna_utils.NeptuneCallback(
                                                    run,
                                                    plots_update_freq=2,  # create/log plots every 2 trials in the documentation its iteration
                                                    log_plot_slice=False,  # do not create/log plot_slice
                                                    log_plot_contour=False,  # do not create/log plot_contour
                                                )

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, callbacks=[neptune_callback],show_progress_bar=True)



[I 2024-04-29 12:54:46,001] A new study created in memory with name: no-name-aaf6acc1-7c30-4e04-90aa-50a858533408


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[I 2024-04-29 12:58:28,834] Trial 0 finished with value: 0.8555328201980089 and parameters: {'colsample_bytree': 0.7597328361507293, 'colsample_bynode': 0.9594957372672832, 'n_estimators': 293, 'max_depth': 19, 'learning_rate': 0.034706746158588604, 'lambda_l1': 7.393947371811905, 'lambda_l2': 9.888266908009667, 'num_leaves': 227, 'min_data_in_leaf': 409}. Best is trial 0 with value: 0.8555328201980089.


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_iter` in params. Will use it instead of argument

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



...

[I 2024-04-29 17:01:37,927] Trial 47 finished with value: 0.8585061943214547 and parameters: {'colsample_bytree': 0.665060460926714, 'colsample_bynode': 0.7114454455815685, 'n_estimators': 796, 'max_depth': 12, 'learning_rate': 0.01892210481879811, 'lambda_l1': 0.560659998928166, 'lambda_l2': 8.473807276924727, 'num_leaves': 328, 'min_data_in_leaf': 469}. Best is trial 43 with value: 0.8594579931525512.


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_iter` in params. Will use it instead of argument

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



[I 2024-04-29 17:09:17,287] Trial 48 finished with value: 0.8576270137642998 and parameters: {'colsample_bytree': 0.6972761867871807, 'colsample_bynode': 0.7344923643488029, 'n_estimators': 721, 'max_depth': 13, 'learning_rate': 0.0141189221415406, 'lambda_l1': 1.205021375584628, 'lambda_l2': 9.218230616206075, 'num_leaves': 405, 'min_data_in_leaf': 440}. Best is trial 43 with value: 0.8594579931525512.


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_iter` in params. Will use it instead of argument

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



[I 2024-04-29 17:14:31,974] Trial 49 finished with value: 0.8535827817097044 and parameters: {'colsample_bytree': 0.6886968643971931, 'colsample_bynode': 0.6663435449730664, 'n_estimators': 721, 'max_depth': 9, 'learning_rate': 0.010578137921350348, 'lambda_l1': 2.143607403034903, 'lambda_l2': 8.351390546726725, 'num_leaves': 367, 'min_data_in_leaf': 502}. Best is trial 43 with value: 0.8594579931525512.


In [55]:
# Now getting the best best parameters
best_params = study.best_params
best_params.update({
                    "n_jobs": -1,
                    "n_iter": 100
                })

# Now model training
model = lgb.LGBMClassifier(**best_params)
model.fit(x_train, y_train)

# NOW roc score
y_pred_prob = model.predict_proba(x_test)
y_pred = model.predict(x_test)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))
run["test_roc"] = roc_auc_score(y_test, y_pred)
# run.stop()

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning:

Found `n_iter` in params. Will use it instead of argument



[LightGBM] [Warning] min_data_in_leaf is set=437, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=437
[LightGBM] [Warning] lambda_l2 is set=7.596840201631312, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.596840201631312
[LightGBM] [Warning] lambda_l1 is set=2.07724257197326, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.07724257197326
[LightGBM] [Warning] num_iterations is set=100, n_iter=100 will be ignored. Current value: num_iterations=100
[LightGBM] [Warning] min_data_in_leaf is set=437, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=437
[LightGBM] [Warning] lambda_l2 is set=7.596840201631312, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.596840201631312
[LightGBM] [Warning] lambda_l1 is set=2.07724257197326, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.07724257197326
[LightGBM] [Info] Number of positive: 36026, number of negative: 1108968
[LightGBM] [Info] Auto-choosing col-wise multi-

In [67]:
features = x_train.columns


In [66]:
d_type = x_train.dtypes

In [68]:
# prompt: create a pandas df with above two lists features and d_type and save them into csv file

import pandas as pd

data = {'features': features, 'data_type': d_type}

df = pd.DataFrame(data)

df.to_csv('data_types.csv', index=False)


In [71]:
run["training_featues_type"].upload("data_types.csv")

### Prediction

In [56]:
X_test = df_test.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(X_test)[:, 1], index=X_test.index)

ValueError: Number of features of the model must match the input. Model n_features_ is 399 and input n_features is 425

### Submission

In [ ]:
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred

In [ ]:
print("Check null: ", df_subm["score"].isnull().any())

df_subm.head()

In [ ]:
df_subm.to_csv("submission.csv")

In [72]:
run.stop()


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kaggle-housing/home-depot/e/HOM-19/metadata
